In [1]:
!pip install --upgrade --no-cache-dir gdown

!pip install -q hazm
!pip install -q parsivar

# !pip install -q datasets
!pip install -q transformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.3 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.4.5 which is incompatible.
hazm 0.7.0 requires nltk==3.3, but you have nltk 3.4.5 which is incompatible.


In [2]:
!pip install -q "datasets==2.10.1" # previously 2.9.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.79.0 which is incompatible.
tfx-bsl 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 11.0.0 which is incompatible.
tensorflow-transform 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 11.0.0 which is incompatible.
apache-beam 2.44.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
apache-beam 2.44.0 requires pyarrow<10.0.0,>=0.15.1, but you have pyarrow 11.0.0 which is incompatible.


In [1]:
import datasets
datasets.__version__

'2.10.1'

In [2]:
from IPython.display import FileLink
FileLink('torob_v4-ParsBERT-model_3-epoch_8.w')

/kaggle/working/torob_v4-ParsBERT-model_3-epoch_8.w

In [9]:
!rm -rf "/kaggle/working/dataset_v3_2_model_2.zip" "/kaggle/working/torobv3_2-ParsBERT-model_1-epoch_3.w" "/kaggle/working/torobv3_2-ParsBERT-model_1-epoch_5.w"

In [10]:
# !gdown 1mWMpLJ1oqaAGbKiPBytLnu5B87r4crAg
# !gdown "1UqQfJH4TTk-CaKsraTyRzgtm3b1VXbN_" #train_test dataset version 4
!gdown "14Ul3ZktHpDHB7oUe2S7v5RtijeXbhFwx"

Downloading...
From: https://drive.google.com/uc?id=14Ul3ZktHpDHB7oUe2S7v5RtijeXbhFwx
To: /kaggle/working/torob_v4-ParsBERT-model_3-epoch_5.w
100%|█████████████████████████████████████████| 652M/652M [00:03<00:00, 163MB/s]


In [9]:
!7z x -y /kaggle/working/train_test_dataset_v4.7z 


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=C.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /kaggle/workin                        1 file, 16818016 bytes (17 MiB)

Extracting archive: /kaggle/working/train_test_dataset_v4.7z
--
Path = /kaggle/working/train_test_dataset_v4.7z
Type = 7z
Physical Size = 16818016
Headers Size = 288
Method = LZMA2:26
Solid = +
Blocks = 1

      9% 3 - test_dataset_v4/dataset_info.js                                         21% 5 - train_dataset_v4/data-00000-of-00001.arro                                                   30% 5 - train_dataset_v4/data-00000-of-00001.arro                                                   41% 5 - train_dataset_v4/data-00000-of-00001.arro                                                   56% 5 - train_dataset_v4/data-00000-of-00001.arro                                                   7

In [10]:
!mv  "/kaggle/working/test_dataset_v4" "test_dataset"

In [11]:
!mv  "/kaggle/working/train_dataset_v4" "train_dataset"

In [10]:
# !rm -rf "data-00000-of-00001.arrow" "dataset_info.json" "train_dataset_v4/" "test_dataset_v4/" "state.json"

In [2]:
# !unzip dataset_v3_2_model_2.zip

In [3]:
from IPython.display import FileLink
# FileLink('torobv3_2-ParsBERT-model_1-epoch_7.w')
# FileLink('torob_v4-ParsBERT-model_3-epoch_1.w')
FileLink('torob_v4-ParsBERT-model_3-epoch_7.w')

/kaggle/working/torob_v4-ParsBERT-model_3-epoch_7.w

In [4]:
import csv
import json
import sqlite3

from collections import defaultdict, Counter

import hazm
from parsivar import Normalizer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm_notebook

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import torch
from torch import nn

from datasets import Dataset, DatasetDict, load_dataset, load_metric, load_from_disk, concatenate_datasets
from transformers import AutoTokenizer, AutoModel
from transformers import AdamW

In [5]:
ls

__notebook_source__.ipynb            torob_v4-ParsBERT-model_3-epoch_7.w
test_dataset/                        torob_v4-ParsBERT-model_3-epoch_8.w
torob_v4-ParsBERT-model_3-epoch_5.w  train_dataset/
torob_v4-ParsBERT-model_3-epoch_6.w  train_test_dataset_v4.7z


## Model

In [6]:
class BaseModel(nn.Module):
    def __init__(self, c_model):
        super(BaseModel, self).__init__()
        self.bert = AutoModel.from_pretrained(c_model)
        self.fc = nn.Linear(self.bert.config.hidden_size, 256)
        self.relu = nn.ReLU()
        # self.dropout = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(256, 1)
        
    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        out = self.fc(output[1])
        return self.fc2(self.relu(out))


In [7]:
def validation(model, val_loader, loss_fn, test=True, device="cuda"):
    m = nn.Sigmoid()
    val_loss = 0.0
    model.eval()
    reals = []
    preds_list = []
    for batch in val_loader:
        input_ids = batch["input_ids"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"]
        with torch.no_grad():
            preds = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
            )
        loss = loss_fn(preds.squeeze(1), labels.to(device).to(torch.float32))
        val_loss += loss.item()
    val_loss = val_loss / len(val_loader)
    return val_loss


In [8]:
def train(
    model, optimizer, loss_fn, train_loader, test_loader, epochs=1, device="cuda"
):
    m = nn.Sigmoid()
    model.to(device)
    for epoch in range(epochs):
        training_loss = 0.0
        val_loss = 0.0
        model.train()
        t_labels = []
        t_preds = []
        batch_idx = 0
        for batch in tqdm_notebook(train_loader):
            input_ids = batch["input_ids"].to(device)
            token_type_ids = batch["token_type_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"]
            t_labels += labels.numpy().tolist()
            optimizer.zero_grad()
            preds = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
            )
            loss = loss_fn(preds.squeeze(1), labels.to(device).to(torch.float32))
            loss.backward()
            optimizer.step()
            training_loss += loss.item()
            ####################################
            if batch_idx % 500 == 0:
                print(
                    "Epoch: {}, Batch: {}, Loss: {:.4f}".format(
                    # Val Loss: {:.4f}".format(
                        epoch,
                        batch_idx,
                        training_loss / (batch_idx + 1),
                        # validation(model, test_loader, loss_fn, device=device),
                    )
                )
            batch_idx += 1
            ####################################
        training_loss = training_loss / len(train_loader)
        print("Epoch: {},training loss: {:.4f}  ".format(epoch, training_loss))
    test_loss = validation(model, test_loader, loss_fn, device=device)
    print("test loss: {:.2f} ".format(test_loss))


In [9]:
def collate_batch(batch):
    input_ids = []
    attention_mask = []
    token_type_ids = []
    labels = []
    for b in batch:
        input_ids.append(b["input_ids"])
        token_type_ids.append(b["token_type_ids"])
        attention_mask.append(b["attention_mask"])
        labels.append(b["label"])
    input_ids = torch.nn.utils.rnn.pad_sequence(
        input_ids, batch_first=True, padding_value=0.0
    )
    attention_mask = torch.nn.utils.rnn.pad_sequence(
        attention_mask, batch_first=True, padding_value=0.0
    )
    token_type_ids = torch.nn.utils.rnn.pad_sequence(
        token_type_ids, batch_first=True, padding_value=0.0
    )
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "token_type_ids": token_type_ids,
        "label": torch.Tensor(labels).to(torch.long),
    }

In [10]:
c_model = "HooshvareLab/bert-fa-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(c_model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BaseModel(c_model)
model = nn.DataParallel(model)
model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-6)
loss_fn = nn.MSELoss()

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation

In [13]:
# !gdown "1ldOePYFSSpnEypj8Qj4FsVtMGizWpi4-"

In [14]:
# model.load_state_dict(
#     torch.load("/kaggle/working/torobv3_2-ParsBERT-model_1-epoch_2.w"),
#     strict=True,
# )
# model.to(device);

In [11]:
model.load_state_dict(
    torch.load("/kaggle/working/torob_v4-ParsBERT-model_3-epoch_8.w"),
    strict=True,
)
model.to(device);

## Load Data

In [12]:
train_dataset = Dataset.load_from_disk("./train_dataset")
test_dataset = Dataset.load_from_disk("./test_dataset")

In [13]:
train_dataset = train_dataset.shuffle()
train_dataset.set_format(type='torch')
test_dataset.set_format(type='torch')

train_dataset, test_dataset

(Dataset({
     features: ['input_ids', 'attention_mask', 'token_type_ids', 'label'],
     num_rows: 543273
 }),
 Dataset({
     features: ['input_ids', 'attention_mask', 'token_type_ids', 'label'],
     num_rows: 60364
 }))

In [14]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32, collate_fn=collate_batch, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=32, collate_fn=collate_batch, shuffle=True
)

In [15]:
train(model, optimizer, loss_fn, train_loader, test_loader, epochs=1, device="cuda")

  0%|          | 0/16978 [00:00<?, ?it/s]

Epoch: 0, Batch: 0, Loss: 1.4803
Epoch: 0, Batch: 500, Loss: 1.5028
Epoch: 0, Batch: 1000, Loss: 1.5690
Epoch: 0, Batch: 1500, Loss: 1.5552
Epoch: 0, Batch: 2000, Loss: 1.5645
Epoch: 0, Batch: 2500, Loss: 1.5556
Epoch: 0, Batch: 3000, Loss: 1.5738
Epoch: 0, Batch: 3500, Loss: 1.5863
Epoch: 0, Batch: 4000, Loss: 1.5965
Epoch: 0, Batch: 4500, Loss: 1.5876
Epoch: 0, Batch: 5000, Loss: 1.5799
Epoch: 0, Batch: 5500, Loss: 1.5838
Epoch: 0, Batch: 6000, Loss: 1.5891
Epoch: 0, Batch: 6500, Loss: 1.5896
Epoch: 0, Batch: 7000, Loss: 1.5954
Epoch: 0, Batch: 7500, Loss: 1.5957
Epoch: 0, Batch: 8000, Loss: 1.6019
Epoch: 0, Batch: 8500, Loss: 1.6031
Epoch: 0, Batch: 9000, Loss: 1.6053
Epoch: 0, Batch: 9500, Loss: 1.6080
Epoch: 0, Batch: 10000, Loss: 1.6105
Epoch: 0, Batch: 10500, Loss: 1.6141
Epoch: 0, Batch: 11000, Loss: 1.6131
Epoch: 0, Batch: 11500, Loss: 1.6110
Epoch: 0, Batch: 12000, Loss: 1.6077
Epoch: 0, Batch: 12500, Loss: 1.6056
Epoch: 0, Batch: 13000, Loss: 1.6094
Epoch: 0, Batch: 13500, L

In [16]:
torch.save(
    model.state_dict(),
    "/kaggle/working/" + "torob_v4-ParsBERT-model_3-epoch_9" + ".w",
)

In [17]:
FileLink('/kaggle/working/torob_v4-ParsBERT-model_3-epoch_9.w')

/kaggle/working/torob_v4-ParsBERT-model_3-epoch_9.w